In [22]:
import os
import numpy as np
import pandas as pd
import librosa
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments


In [23]:
# data_dir="C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small"
# file_path="C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester 6/460j/final_project_data/fma_small/000/000002.mp3"
train_path = 'C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester6/460j/460jFinalProject/extracted_features_combined.csv'
# train_path = 'C:/Users/divya/OneDrive/Documents/UTAUSTIN/Semester6/460j/460jFinalProject/extracted_features_wav2vec.csv'


In [24]:
data_df=pd.read_csv(train_path, index_col=0)
# data_df=pd.read_csv(train_path)

In [25]:
display(data_df.head())

,0,1,2,3,4,5,6,7,8,9,...,1176,1177,1178,1179,1180,1181,1182,1183,1184,genre
track_id,,,,,,,,,,,,,,,,,,,,,
2,-122.713936,117.760094,-42.334183,38.061005,-23.012323,23.920383,-11.944865,13.687291,-7.294582,4.041143,...,0.000032,0.000033,0.000039,0.000048,0.000078,0.000110,0.000089,0.000050,0.000029,Hip-Hop
5,-162.232422,131.435989,-18.426781,51.631371,-15.444844,23.860094,-7.867664,10.686234,-0.314591,12.864646,...,0.000039,0.000041,0.000060,0.000092,0.000102,0.000146,0.000153,0.000072,0.000038,Hip-Hop
10,-75.631538,154.537613,-63.849487,22.506426,2.425359,10.459981,-13.007332,9.818939,-7.640783,3.136315,...,0.000056,0.000066,0.000069,0.000083,0.000095,0.000103,0.000103,0.000072,0.000058,Pop
140,-322.997864,133.684906,15.497437,54.755615,8.718211,37.022747,-2.116411,15.265619,6.175477,-2.599205,...,0.000041,0.000035,0.000031,0.000038,0.000050,0.000052,0.000034,0.000024,0.000022,Folk
141,-253.314560,157.769379,-6.081675,26.920168,6.813843,15.842272,-12.930041,10.373278,-0.609945,-8.300909,...,0.000029,0.000025,0.000026,0.000048,0.000095,0.000081,0.000036,0.000029,0.000023,Folk


In [26]:

X = StandardScaler().fit_transform(data_df.drop('genre', axis=1))
y = pd.factorize(data_df['genre'])[0]  # Convert genre to numerical labels


In [27]:
display(y)

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [29]:
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

X_train_tensor_rnn = X_train_tensor.unsqueeze(1).expand(-1, 10, -1)  # Shape becomes [batch_size, 1, num_features]
X_test_tensor_rnn = X_test_tensor.unsqueeze(1).expand(-1, 10, -1)  # Shape becomes [batch_size, 1, num_features]
train_dataset_rnn = TensorDataset(X_train_tensor_rnn, y_train_tensor)
test_dataset_rnn = TensorDataset(X_test_tensor_rnn, y_test_tensor)
train_loader_rnn = DataLoader(train_dataset_rnn, batch_size=256, shuffle=True)
test_loader_rnn = DataLoader(test_dataset_rnn, batch_size=256, shuffle=False)


In [30]:
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 2048),
            nn.LeakyReLU(),
            nn.Dropout(0.1),
            nn.Linear(2048, 1024),
            nn.LeakyReLU(),
            nn.Dropout(0.1),
            nn.Linear(1024, 512),
            nn.LeakyReLU(),
            nn.Dropout(0.05),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(),
            nn.Dropout(0.05),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Linear(64, output_dim)
        )
        
    def forward(self, x):
        return self.layers(x)


In [31]:
class CNN1D(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(CNN1D, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv1d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(32 * (input_dim // 4), 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
        
    def forward(self, x):
        x = x.unsqueeze(1)  # Add a channel dimension
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # Flatten
        return self.fc_layers(x)


In [32]:
class RNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128, num_layers=1):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x, _ = self.rnn(x)
        x = self.fc(x[:, -1, :])  # Take the last time step
        return x

class LSTM(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=128, num_layers=1):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x, (hn, cn) = self.lstm(x)
        x = self.fc(x[:, -1, :])  # Take the last time step
        return x


In [33]:
class EnsembleModel(nn.Module):
    def __init__(self, mlp, cnn, rnn, lstm, output_dim):
        super(EnsembleModel, self).__init__()
        self.mlp = mlp
        self.cnn = cnn
        self.rnn = rnn
        self.lstm = lstm
        self.fc = nn.Linear(output_dim * 4, output_dim)  # Assuming each model outputs the class probabilities
    
    def forward(self, x):
        out_mlp = self.mlp(x)
        out_cnn = self.cnn(x)
        out_rnn = self.rnn(x.unsqueeze(1))  # Assume RNN/LSTM expects sequence data
        out_lstm = self.lstm(x.unsqueeze(1))
        
        # Concatenate along the feature dimension
        combined = torch.cat((out_mlp, out_cnn, out_rnn, out_lstm), dim=1)
        output = self.fc(combined)
        return output


In [34]:
def train_model(model, train_loader, test_loader, device, epochs=100):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    acc=0

    for epoch in range(epochs):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        model.eval()
        total = correct = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        acc= max(acc,100 * correct / total)

        print(f'Epoch {epoch+1}: Accuracy: {100 * correct / total}%')
    return(acc)

In [35]:
input_size = X_train.shape[1]
num_classes = len(np.unique(y))

In [36]:
model_mlp=MLP(input_size,num_classes).to(device)
train_model(model_mlp, train_loader, test_loader, device)
# 54.127063531765884 entropy loss

Epoch 1: Accuracy: 16.808404202101052%
Epoch 2: Accuracy: 21.710855427713856%
Epoch 3: Accuracy: 25.6128064032016%
Epoch 4: Accuracy: 26.963481740870435%
Epoch 5: Accuracy: 28.31415707853927%
Epoch 6: Accuracy: 28.564282141070535%
Epoch 7: Accuracy: 26.313156578289146%
Epoch 8: Accuracy: 34.167083541770886%
Epoch 9: Accuracy: 35.41770885442721%
Epoch 10: Accuracy: 33.41670835417709%
Epoch 11: Accuracy: 36.86843421710856%
Epoch 12: Accuracy: 38.06903451725863%
Epoch 13: Accuracy: 39.96998499249625%
Epoch 14: Accuracy: 38.36918459229615%
Epoch 15: Accuracy: 40.120060030015004%
Epoch 16: Accuracy: 38.669334667333665%
Epoch 17: Accuracy: 43.221610805402705%
Epoch 18: Accuracy: 39.86993496748374%
Epoch 19: Accuracy: 42.72136068034017%
Epoch 20: Accuracy: 42.92146073036518%
Epoch 21: Accuracy: 43.72186093046523%
Epoch 22: Accuracy: 41.470735367683844%
Epoch 23: Accuracy: 45.42271135567784%
Epoch 24: Accuracy: 45.872936468234116%
Epoch 25: Accuracy: 42.821410705352676%
Epoch 26: Accuracy: 44.

50.675337668834416

In [37]:
model_cnn=CNN1D(input_size,num_classes).to(device)
train_model(model_cnn, train_loader, test_loader, device) 
# 51.52576288144072 entropy loss

c:\Users\divya\OneDrive\Documents\UTAUSTIN\Semester6\460j\460jFinalProject\.venv\Lib\site-packages\torch\nn\modules\conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv1d(input, weight, bias, self.stride,


Epoch 1: Accuracy: 35.56778389194597%
Epoch 2: Accuracy: 41.57078539269635%
Epoch 3: Accuracy: 45.82291145572786%
Epoch 4: Accuracy: 45.222611305652826%
Epoch 5: Accuracy: 48.124062031015505%
Epoch 6: Accuracy: 48.37418709354677%
Epoch 7: Accuracy: 47.1735867933967%
Epoch 8: Accuracy: 47.67383691845923%
Epoch 9: Accuracy: 49.27463731865933%
Epoch 10: Accuracy: 47.22361180590295%
Epoch 11: Accuracy: 46.923461730865434%
Epoch 12: Accuracy: 45.722861430715355%
Epoch 13: Accuracy: 47.723861930965484%
Epoch 14: Accuracy: 48.32416208104052%
Epoch 15: Accuracy: 45.37268634317159%
Epoch 16: Accuracy: 48.274137068534266%
Epoch 17: Accuracy: 46.22311155577789%
Epoch 18: Accuracy: 46.273136568284144%
Epoch 19: Accuracy: 45.32266133066533%
Epoch 20: Accuracy: 47.77388694347174%
Epoch 21: Accuracy: 47.52376188094047%
Epoch 22: Accuracy: 46.37318659329665%
Epoch 23: Accuracy: 45.872936468234116%
Epoch 24: Accuracy: 46.923461730865434%
Epoch 25: Accuracy: 46.97348674337169%
Epoch 26: Accuracy: 47.073

49.27463731865933

In [38]:
model_rnn=RNN(input_size,num_classes).to(device)
train_model(model_rnn, train_loader_rnn, test_loader_rnn, device) 
#50.17508754377189 entropy loss

Epoch 1: Accuracy: 33.01650825412706%
Epoch 2: Accuracy: 35.71785892946473%
Epoch 3: Accuracy: 37.91895947973987%
Epoch 4: Accuracy: 40.020010005002504%
Epoch 5: Accuracy: 39.96998499249625%
Epoch 6: Accuracy: 39.36968484242121%
Epoch 7: Accuracy: 41.97098549274637%
Epoch 8: Accuracy: 42.12106053026513%
Epoch 9: Accuracy: 43.1215607803902%
Epoch 10: Accuracy: 40.97048524262131%
Epoch 11: Accuracy: 41.97098549274637%
Epoch 12: Accuracy: 44.17208604302151%
Epoch 13: Accuracy: 43.92196098049025%
Epoch 14: Accuracy: 42.77138569284642%
Epoch 15: Accuracy: 43.92196098049025%
Epoch 16: Accuracy: 43.27163581790895%
Epoch 17: Accuracy: 42.271135567783894%
Epoch 18: Accuracy: 44.17208604302151%
Epoch 19: Accuracy: 43.57178589294647%
Epoch 20: Accuracy: 43.9719859929965%
Epoch 21: Accuracy: 43.071535767883944%
Epoch 22: Accuracy: 42.92146073036518%
Epoch 23: Accuracy: 45.17258629314657%
Epoch 24: Accuracy: 43.57178589294647%
Epoch 25: Accuracy: 43.92196098049025%
Epoch 26: Accuracy: 43.3716858429

45.17258629314657

In [39]:
model_lstm=LSTM(input_size,num_classes).to(device)
train_model(model_lstm, train_loader_rnn, test_loader_rnn, device) 
#49.67483741870935 entropy loss

Epoch 1: Accuracy: 35.56778389194597%
Epoch 2: Accuracy: 38.46923461730866%
Epoch 3: Accuracy: 39.319659829914954%
Epoch 4: Accuracy: 41.5207603801901%
Epoch 5: Accuracy: 40.07003501750876%
Epoch 6: Accuracy: 39.66983491745873%
Epoch 7: Accuracy: 40.22011005502751%
Epoch 8: Accuracy: 40.17008504252126%
Epoch 9: Accuracy: 40.62031015507754%
Epoch 10: Accuracy: 43.27163581790895%
Epoch 11: Accuracy: 42.22111055527764%
Epoch 12: Accuracy: 42.97148574287144%
Epoch 13: Accuracy: 42.4712356178089%
Epoch 14: Accuracy: 43.67183591795898%
Epoch 15: Accuracy: 42.32116058029015%
Epoch 16: Accuracy: 44.62231115557779%
Epoch 17: Accuracy: 44.57228614307154%
Epoch 18: Accuracy: 43.82191095547774%
Epoch 19: Accuracy: 44.272136068034015%
Epoch 20: Accuracy: 45.02251125562781%
Epoch 21: Accuracy: 42.021010505252626%
Epoch 22: Accuracy: 42.77138569284642%
Epoch 23: Accuracy: 42.57128564282141%
Epoch 24: Accuracy: 40.920460230115054%
Epoch 25: Accuracy: 44.7223611805903%
Epoch 26: Accuracy: 43.5217608804

46.07303651825913

In [40]:
model_cnn_2=CNN1D(input_size,num_classes).to(device)
model_mlp_2=MLP(input_size,num_classes).to(device)
model_rnn_2=RNN(input_size,num_classes).to(device)
model_lstm_2=LSTM(input_size,num_classes).to(device)

model_ens=EnsembleModel(mlp=model_mlp_2,cnn=model_cnn_2,rnn=model_rnn_2,lstm=model_lstm_2,output_dim=num_classes).to(device)
train_model(model_ens, train_loader, test_loader, device)
# 53.62681340670335 accuracy entropy loss

Epoch 1: Accuracy: 34.46723361680841%
Epoch 2: Accuracy: 43.17158579289645%
Epoch 3: Accuracy: 44.022011005502755%
Epoch 4: Accuracy: 45.222611305652826%
Epoch 5: Accuracy: 46.17308654327164%
Epoch 6: Accuracy: 46.77338669334667%
Epoch 7: Accuracy: 48.47423711855928%
Epoch 8: Accuracy: 47.02351175587794%
Epoch 9: Accuracy: 46.77338669334667%
Epoch 10: Accuracy: 45.32266133066533%
Epoch 11: Accuracy: 45.6728364182091%
Epoch 12: Accuracy: 46.523261630815405%
Epoch 13: Accuracy: 45.97298649324662%
Epoch 14: Accuracy: 46.37318659329665%
Epoch 15: Accuracy: 44.57228614307154%
Epoch 16: Accuracy: 46.17308654327164%
Epoch 17: Accuracy: 45.222611305652826%
Epoch 18: Accuracy: 44.422211105552776%
Epoch 19: Accuracy: 45.12256128064032%
Epoch 20: Accuracy: 44.672336168084044%
Epoch 21: Accuracy: 46.62331165582791%
Epoch 22: Accuracy: 44.822411205602805%
Epoch 23: Accuracy: 44.52226113056528%
Epoch 24: Accuracy: 42.77138569284642%
Epoch 25: Accuracy: 43.621810905452726%
Epoch 26: Accuracy: 44.8724

48.47423711855928

In [41]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import random

input_size = X_train.shape[1]
output_size = len(np.unique(y))

In [42]:
import torch
from torch.utils.data import DataLoader

def train_model(model, optimizer, train_loader, test_loader, device, epochs=50):
    criterion = torch.nn.CrossEntropyLoss()  # Assuming classification task
    best_acc = 0
    best_epoch = 0
    
    for epoch in range(epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        total_train = 0
        correct_train = 0
        
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()  # Zero the parameter gradients
            outputs = model(inputs)  # Forward pass
            loss = criterion(outputs, labels)  # Calculate loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            
            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_train / total_train * 100
        
        # Validation phase
        model.eval()  # Set model to evaluate mode
        val_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():  # No gradient calculation for validation
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_loss /= len(test_loader.dataset)
        val_accuracy = correct / total * 100
        
        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'Train Loss: {epoch_loss:.4f} - Train Acc: {epoch_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f} - Val Acc: {val_accuracy:.2f}%')
        
        # Save the model if the validation accuracy is the best we've seen so far.
        if val_accuracy > best_acc:
            best_acc = val_accuracy
            best_epoch = epoch
            torch.save(model.state_dict(), 'best_model.pth')  # Save best model
            
    print(f'Best Validation Accuracy: {best_acc:.2f}% at Epoch {best_epoch+1}')
    return best_acc


In [43]:
import torch
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR  # Learning rate scheduler

def train_model_best(model, optimizer, train_loader, test_loader, device, epochs=50):
    criterion = torch.nn.CrossEntropyLoss()
    scheduler = StepLR(optimizer, step_size=30, gamma=0.1)  # Reduce LR every 30 epochs
    best_acc = 0
    best_epoch = 0
    patience = 10  # Early stopping patience
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        total_train = 0
        correct_train = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_train / total_train * 100

        # Validation phase
        model.eval()
        val_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_loss /= len(test_loader.dataset)
        val_accuracy = correct / total * 100

        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'Train Loss: {epoch_loss:.4f} - Train Acc: {epoch_acc:.2f}%')
        print(f'Val Loss: {val_loss:.4f} - Val Acc: {val_accuracy:.2f}%')

        # Learning rate scheduler step
        scheduler.step()

        # Early stopping check
        if val_accuracy > best_acc:
            best_acc = val_accuracy
            best_epoch = epoch
            patience_counter = 0  # Reset counter
            torch.save(model.state_dict(), 'best_model.pth')  # Save best model
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Stopping early at epoch {epoch + 1}")
                break

    print(f'Best Validation Accuracy: {best_acc:.2f}% at Epoch {best_epoch+1}')
    return best_acc


In [19]:


class NeuralNet(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_layers, neurons_start, dropout_rate, use_bn, activation_fn):
        super(NeuralNet, self).__init__()
        layers = []

        # Calculate the decrement step to ensure the last layer has 64 neurons
        if neurons_start > 64:
            decrement = (neurons_start - 64) / (hidden_layers - 1)
        else:
            decrement = 0

        current_neurons = neurons_start

        for i in range(hidden_layers):
            next_neurons = max(64, int(current_neurons - decrement * i))  # Ensure we do not go below 64 neurons
            layers.append(nn.Linear(input_dim, next_neurons))
            if use_bn:
                layers.append(nn.BatchNorm1d(next_neurons))
            layers.append(activation_fn())
            if dropout_rate > 0:
                layers.append(nn.Dropout(dropout_rate))
            input_dim = next_neurons

        layers.append(nn.Linear(next_neurons, output_dim))  # Final layer to output
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)

In [18]:
params = {
    'num_layers': [2, 4, 6,8],  # Number of hidden layers
    'neurons_start': [256, 512, 1024,2048],  # Starting number of neurons
    'dropout_rate': [0.05, 0.1, 0.2],
    'batch_size': [64, 128, 256],
    'use_bn': [True, False],
    'learning_rate': [1e-2, 1e-3, 1e-4],
    'optimizer': [torch.optim.Adam, torch.optim.SGD],
    'activation_fn': [nn.ReLU, nn.LeakyReLU, nn.ELU],

}

# Random Search
num_searches = 256  # Number of random configurations to try
best_acc = 0
best_params = None
best_model = None

In [25]:
for _ in range(num_searches):  # Number of configurations to try
    chosen_params = {k: random.choice(v) for k, v in params.items()}
    
    # Create model
    model = NeuralNet(input_size, num_classes, chosen_params['num_layers'], chosen_params['neurons_start'],
                      chosen_params['dropout_rate'], chosen_params['use_bn'], chosen_params['activation_fn']).to(device)
    
    # Setup optimizer
    optimizer = chosen_params['optimizer'](model.parameters(), lr=chosen_params['learning_rate'])
    
    # Train the model (function to be defined based on your setup)
    acc = train_model(model, optimizer, train_loader, test_loader, device)
    # acc= train_model_best(model, optimizer, train_loader, test_loader, device)
    # Track the best model
    if acc > best_acc:
        best_acc = acc
        best_params = chosen_params
        best_model = model_mlp

Epoch 1/50:
Train Loss: 2.0729 - Train Acc: 16.90%
Val Loss: 2.0674 - Val Acc: 13.56%
Epoch 2/50:
Train Loss: 1.9768 - Train Acc: 23.34%
Val Loss: 1.9398 - Val Acc: 29.51%
Epoch 3/50:
Train Loss: 1.9073 - Train Acc: 29.46%
Val Loss: 1.8503 - Val Acc: 34.57%
Epoch 4/50:
Train Loss: 1.8486 - Train Acc: 32.21%
Val Loss: 1.7979 - Val Acc: 36.67%
Epoch 5/50:
Train Loss: 1.7983 - Train Acc: 35.40%
Val Loss: 1.7417 - Val Acc: 38.72%
Epoch 6/50:
Train Loss: 1.7611 - Train Acc: 36.21%
Val Loss: 1.7210 - Val Acc: 38.82%
Epoch 7/50:
Train Loss: 1.7273 - Train Acc: 38.02%
Val Loss: 1.6931 - Val Acc: 39.87%
Epoch 8/50:
Train Loss: 1.6935 - Train Acc: 40.13%
Val Loss: 1.6673 - Val Acc: 39.62%
Epoch 9/50:
Train Loss: 1.6609 - Train Acc: 40.52%
Val Loss: 1.6373 - Val Acc: 40.77%
Epoch 10/50:
Train Loss: 1.6299 - Train Acc: 42.40%
Val Loss: 1.6268 - Val Acc: 40.72%
Epoch 11/50:
Train Loss: 1.6011 - Train Acc: 43.82%
Val Loss: 1.5889 - Val Acc: 42.37%
Epoch 12/50:
Train Loss: 1.5693 - Train Acc: 45.29%


In [26]:

print(f"Best Accuracy: {best_acc}")
print(f"Best Parameters: {best_params}")


# Best Accuracy: 54.17708854427213
# Best Parameters: {'num_layers': 4, 'neurons_start': 2048, 'dropout_rate': 0.1, 'batch_size': 128, 'use_bn': True, 'learning_rate': 0.0001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'activation_fn': <class 'torch.nn.modules.activation.LeakyReLU'>}


Best Accuracy: 54.17708854427213
Best Parameters: {'num_layers': 4, 'neurons_start': 2048, 'dropout_rate': 0.1, 'batch_size': 128, 'use_bn': True, 'learning_rate': 0.0001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'activation_fn': <class 'torch.nn.modules.activation.LeakyReLU'>}


In [44]:
for _ in range(num_searches):  # Number of configurations to try
    chosen_params = {k: random.choice(v) for k, v in params.items()}
    
    # Create model
    model = NeuralNet(input_size, num_classes, chosen_params['num_layers'], chosen_params['neurons_start'],
                      chosen_params['dropout_rate'], chosen_params['use_bn'], chosen_params['activation_fn']).to(device)
    
    # Setup optimizer
    optimizer = chosen_params['optimizer'](model.parameters(), lr=chosen_params['learning_rate'])
    
    # Train the model (function to be defined based on your setup)
    # acc = train_model(model, optimizer, train_loader, test_loader, device)
    acc= train_model_best(model, optimizer, train_loader, test_loader, device)
    # Track the best model
    if acc > best_acc:
        best_acc = acc
        best_params = chosen_params
        best_model = model_mlp

Epoch 1/50:
Train Loss: 1.9873 - Train Acc: 23.59%
Val Loss: 1.9567 - Val Acc: 30.47%
Epoch 2/50:
Train Loss: 1.7650 - Train Acc: 36.00%
Val Loss: 1.7266 - Val Acc: 38.17%
Epoch 3/50:
Train Loss: 1.6560 - Train Acc: 41.85%
Val Loss: 1.6096 - Val Acc: 43.42%
Epoch 4/50:
Train Loss: 1.5678 - Train Acc: 45.87%
Val Loss: 1.5538 - Val Acc: 45.62%
Epoch 5/50:
Train Loss: 1.4977 - Train Acc: 49.21%
Val Loss: 1.5740 - Val Acc: 44.67%
Epoch 6/50:
Train Loss: 1.4261 - Train Acc: 52.43%
Val Loss: 1.5148 - Val Acc: 46.62%
Epoch 7/50:
Train Loss: 1.3709 - Train Acc: 54.03%
Val Loss: 1.5060 - Val Acc: 47.37%
Epoch 8/50:
Train Loss: 1.3169 - Train Acc: 56.83%
Val Loss: 1.5024 - Val Acc: 47.02%
Epoch 9/50:
Train Loss: 1.2744 - Train Acc: 58.72%
Val Loss: 1.5217 - Val Acc: 46.57%
Epoch 10/50:
Train Loss: 1.2267 - Train Acc: 60.67%
Val Loss: 1.4574 - Val Acc: 49.12%
Epoch 11/50:
Train Loss: 1.1793 - Train Acc: 62.35%
Val Loss: 1.4602 - Val Acc: 49.32%
Epoch 12/50:
Train Loss: 1.1238 - Train Acc: 64.34%


In [45]:

print(f"Best Accuracy: {best_acc}")
print(f"Best Parameters: {best_params}")

Best Accuracy: 54.82741370685342
Best Parameters: {'num_layers': 2, 'neurons_start': 512, 'dropout_rate': 0.1, 'batch_size': 64, 'use_bn': False, 'learning_rate': 0.001, 'optimizer': <class 'torch.optim.adam.Adam'>, 'activation_fn': <class 'torch.nn.modules.activation.LeakyReLU'>}


In [46]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# Setup the parameter grid
param_dist_knn = {
    'n_neighbors': randint(1, 30),  # Increased range for number of neighbors
    'weights': ['uniform', 'distance'],  # Weight functions
    'metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev'],  # Distance metrics
    'p': randint(1, 5)  # Power parameter for the Minkowski metric
}
# Initialize the KNN model
knn = KNeighborsClassifier()

# Setup RandomizedSearchCV
random_search_knn = RandomizedSearchCV(knn, param_distributions=param_dist_knn, n_iter=25, cv=5, verbose=1, random_state=42, n_jobs=8)

# Fit the model
random_search_knn.fit(X_train, y_train)

# Best parameters and score
print("Best parameters found for KNN:", random_search_knn.best_params_)
print("Best cross-validated score achieved for KNN:", random_search_knn.best_score_)


Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best parameters found for KNN: {'metric': 'manhattan', 'n_neighbors': 8, 'p': 4, 'weights': 'distance'}
Best cross-validated score achieved for KNN: 0.42035029190992496


In [47]:
from sklearn.metrics import accuracy_score

# Use the best estimator directly from the RandomizedSearchCV results for KNN
best_knn = random_search_knn.best_estimator_

# Predict on the test set
y_pred_knn = best_knn.predict(X_test)

# Calculate accuracy
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"KNN Model Test Accuracy: {accuracy_knn * 100:.2f}%")


KNN Model Test Accuracy: 43.02%


In [48]:
from sklearn.svm import SVC
from scipy.stats import expon, randint

# Setup the parameter grid
param_dist_svm = {
    'C': expon(scale=100),  # Log-uniform range for C
    'kernel': ['linear', 'rbf', 'poly', 'sigmoid'],  # Including sigmoid kernel
    'gamma': expon(scale=0.1),  # Log-uniform range for gamma
    'degree': randint(1, 5)  # Degree of the polynomial kernel, if using poly
}

# Initialize the SVM model
svm = SVC()

# Setup RandomizedSearchCV
random_search_svm = RandomizedSearchCV(svm, param_distributions=param_dist_svm, n_iter=25, cv=5, verbose=1, random_state=42, n_jobs=8)

# Fit the model
random_search_svm.fit(X_train, y_train)

# Best parameters and score
print("Best parameters found for SVM:", random_search_svm.best_params_)
print("Best cross-validated score achieved for SVM:", random_search_svm.best_score_)


Fitting 5 folds for each of 25 candidates, totalling 125 fits


In [ ]:
# Use the best estimator directly from the RandomizedSearchCV results for SVM
best_svm = random_search_svm.best_estimator_

# Predict on the test set
y_pred_svm = best_svm.predict(X_test)

# Calculate accuracy
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"SVM Model Test Accuracy: {accuracy_svm * 100:.2f}%")


In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

# For KNN
print("KNN Confusion Matrix:\n", confusion_matrix(y_test, y_pred_knn))
print("\nKNN Classification Report:\n", classification_report(y_test, y_pred_knn))

# For SVM
print("SVM Confusion Matrix:\n", confusion_matrix(y_test, y_pred_svm))
print("\nSVM Classification Report:\n", classification_report(y_test, y_pred_svm))
